In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.circuit import Gate
from qiskit.visualization import plot_histogram
from typing import List

import numpy as np
import random

from ryQFTAdder import *

# Find the Largest Number (basic ver.)

This part demonstrate the quantum algorithm to solve task 1

## QRAM

Before designing the quantum algorithm that gives the largest number, we have to define how the data in the list can be accessed quantumly. We want a circuit $U_f$ such that:

$$
\ket{i}\ket{0} \xrightarrow{U_f} \ket{i}\ket{f(i)}
$$

where $i$ is the index and $f(i)$ gives the ith number in the list.

For each integer in the list, we choose basic/state encoding, the highest bit is sign bit. For example, 5 and -6 can be encoded as:

$$\begin{align*}
5  &\rightarrow \ket{00101} \\
-6 &\rightarrow \ket{11010}      
\end{align*}$$

4 bits is enough to hold 5 and -6. Since we'll later do addition and subtraction between two numbers in the list, we use 1 extra bit to prevent overflow. The indeices will also be encoded using state encoding and we encode the them into the ctrl bits of mcx gate. The control bit will be controlled by either '0' or '1' based on the state encoding of the index on that position.

In [2]:
"""
Here is a general method to convert a list of number into quantum data.

In this function, we used np.max to find the bound for the list, 
but such information is not encoded into the QRAM so the algorithm that we'll 
present in a moment to solve the problem will not lost any generality. And in practice 
the bound on integer is always predetermined by its type like 'int' in C usually represents 
integers from -2147483648 to 2147483647. The reason that we choose to dynamically determine 
the number of qubit is for compactness and clarity.

"""

def toQRAM(
    data_list: List[int],
    num_data_cbit: int = None
) -> QuantumCircuit:
    """
        returns the circuit U_f such that U_f|i>|0> = |i>|x_i>

        Args:
            data_list:
                the list of number needs to be encoded

            num_data_cbit:
                size of the classical register of for the data qubit
    """
    
    # first construct qram as a gate 

    # define minimum number of qubit to represent all the integers in the list
    bound = np.max(np.abs(data_list))
    num_qubit_i = int(np.ceil(np.log2(len(data_list))))
    num_qubit_xi = len(bin(bound)[2:]) + 2 # one for sign and one for preventing overflow

    if None == num_data_cbit:
        num_data_cbit = num_qubit_xi
    
    # define index and data quantum registers
    reg_i  = QuantumRegister(num_qubit_i)
    reg_xi = QuantumRegister(num_qubit_xi)
    
    qc = QuantumCircuit(reg_i, reg_xi, name="QRAM")

    for i, x in enumerate(data_list):
        # convert index into binary and agree with the number of index qubit
        i_bin = format(i, f'0{num_qubit_i}b') 
        # convert x_i into binary and agree with the number of data qubit
        x_bin = bin(x & int('1' * num_qubit_xi, 2))[2:]

        # determine which control bit is controlled by '0'
        for i_bin_idx, i_bin_symbol in enumerate(reversed(i_bin)):
            if '0' == i_bin_symbol:
                qc.x(reg_i[i_bin_idx])
        
        # encode x_i
        for x_bin_idx, x_bin_symbol in enumerate(reversed(x_bin)):
            if '1' == x_bin_symbol:
                qc.mcx(reg_i, reg_xi[x_bin_idx])
        
        # uncompute for the control bit
        for i_bin_idx, i_bin_symbol in enumerate(reversed(i_bin)):
            if '0' == i_bin_symbol:
                qc.x(reg_i[i_bin_idx])

    # define qram as a gate
    qram_gate = qc.to_gate()

    # using the qram gate, construct the full circuit 
    q_i  = QuantumRegister(num_qubit_i, "q_i")
    q_xi = QuantumRegister(num_qubit_xi, "q_xi")
    c_i = ClassicalRegister(num_qubit_i, "i")
    c_xi = ClassicalRegister(num_data_cbit, "xi") 

    full_circ = QuantumCircuit(q_i, q_xi, c_i, c_xi)
    full_circ.append(qram_gate, range(full_circ.num_qubits))

    return full_circ

For example the list [-3,2,5,-1] in the qram form will be:

In [3]:
ex_list = [-3, 2, 5, 1]

qram_circ = toQRAM(ex_list)
qram_circ.decompose().draw()

┌───┐                    ┌───┐          ┌───┐          ┌───┐     
 q_i_0: ┤ X ├──■────■────■────■──┤ X ├───────■──┤ X ├──■────■──┤ X ├──■──
        ├───┤  │    │    │    │  ├───┤┌───┐  │  ├───┤  │    │  └───┘  │  
 q_i_1: ┤ X ├──■────■────■────■──┤ X ├┤ X ├──■──┤ X ├──■────■─────────■──
        └───┘┌─┴─┐  │    │    │  └───┘└───┘  │  └───┘┌─┴─┐  │       ┌─┴─┐
q_xi_0: ─────┤ X ├──┼────┼────┼──────────────┼───────┤ X ├──┼───────┤ X ├
             └───┘  │    │    │            ┌─┴─┐     └───┘  │       └───┘
q_xi_1: ────────────┼────┼────┼────────────┤ X ├────────────┼────────────
                  ┌─┴─┐  │    │            └───┘          ┌─┴─┐          
q_xi_2: ──────────┤ X ├──┼────┼───────────────────────────┤ X ├──────────
                  └───┘┌─┴─┐  │                           └───┘          
q_xi_3: ───────────────┤ X ├──┼──────────────────────────────────────────
                       └───┘┌─┴─┐                                        
q_xi_4: ────────────────────┤ X ├────────────────────────────────────────
                            └───┘                                        
   i: 2/═════════════════════════════════════════════════════════════════
                                                                         
  xi: 5/═════════════════════════════════════════════════════════════════

And we can confirm the qram indeed stores the disered data by making query on all possible indices:

In [4]:
query_circ = QuantumCircuit(qram_circ.qregs[0])
# equal supperposition on all indices
query_circ.h(query_circ.qregs[0])

# add the query in fron of the qram
query_circ = qram_circ.compose(query_circ, qram_circ.qregs[0], front=True)
# query_circ.append(qAdder(-1, query_circ.qregs[1].size), query_circ.qregs[1])

# measure and get results of the queries
query_circ.measure(query_circ.qregs[0], query_circ.cregs[0])
query_circ.measure(query_circ.qregs[1], query_circ.cregs[1])

query_circ.draw()

┌───┐┌───────┐┌─┐                  
 q_i_0: ┤ H ├┤0      ├┤M├──────────────────
        ├───┤│       │└╥┘┌─┐               
 q_i_1: ┤ H ├┤1      ├─╫─┤M├───────────────
        └───┘│       │ ║ └╥┘┌─┐            
q_xi_0: ─────┤2      ├─╫──╫─┤M├────────────
             │       │ ║  ║ └╥┘┌─┐         
q_xi_1: ─────┤3 QRAM ├─╫──╫──╫─┤M├─────────
             │       │ ║  ║  ║ └╥┘┌─┐      
q_xi_2: ─────┤4      ├─╫──╫──╫──╫─┤M├──────
             │       │ ║  ║  ║  ║ └╥┘┌─┐   
q_xi_3: ─────┤5      ├─╫──╫──╫──╫──╫─┤M├───
             │       │ ║  ║  ║  ║  ║ └╥┘┌─┐
q_xi_4: ─────┤6      ├─╫──╫──╫──╫──╫──╫─┤M├
             └───────┘ ║  ║  ║  ║  ║  ║ └╥┘
   i: 2/═══════════════╩══╩══╬══╬══╬══╬══╬═
                       0  1  ║  ║  ║  ║  ║ 
  xi: 5/═════════════════════╩══╩══╩══╩══╩═
                             0  1  2  3  4

In [5]:
print("Original list: ", ex_list)

# run the circuit to get the result of the queries
simulator = Aer.get_backend('qasm_simulator')
result = execute(query_circ, simulator).result()
counts = result.get_counts()
# raw output in binary
print("Query raw data: ", counts)

# extract original classical data 
print("Decoded result: ")
for xi_i in counts:
    xi_bin, i_bin = xi_i.split(" ")
    
    # have to consider the sign when convert back into integer 
    sign_bit = xi_bin[0]
    xi = int(xi_bin[1:], 2)
    if '1' == sign_bit:
        xi = -(2**(len(xi_bin)-1) - xi)
    # index is unsigned
    i = int(i_bin, 2)

    print(f"f[{i}]: {xi}")

Original list:  [-3, 2, 5, 1]
Query raw data:  {'11101 00': 257, '00101 10': 268, '00001 11': 227, '00010 01': 272}
Decoded result: 
f[0]: -3
f[2]: 5
f[3]: 1
f[1]: 2


In [6]:
counts.most_frequent()

'00010 01'

We can see the recovered data is exactly the same as the original list

## Quantum-classical Adder

With classically given integer X, positive or negative, we want the adder to do:

$$
    \ket{n} \xrightarrow{} \ket{n + X}
$$

There are many different methods, here we choose a implementation in fourier basis. Both the fourier transform defined by y-rotation and the addition circuit in such basis are proposed by Rich Rines & Isaac Chuang^[1]. Detailed implementation is in `ryQFTAdder.py`.

[1]: Rines, R., & Chuang, I. (2018). High Performance Quantum Modular Multipliers. [arXiv:1801.01081](https://arxiv.org/abs/1801.01081).

## Solution

### algorithm

Now we have all the ingredients. Here's the algorithm to find the largest number from a list two integers `[a, b]` :

1. Encode the list into qram.
2. Randomly chooes an index `i`, query classically and let the result be `X`.
3. Consctruct quantum circuit that has the same size as the qram of the list. Apply `H` on the index register.
4. Apply the qram circuit.
5. Apply quantum-classical adder to add `-X` to the data register of the circuit.
6. Measure the index register, get `q_i`. Measure the highest bit, the sign bit, of the data register, get `sx_i`.
7. If `q_i == i`, repeat step 2 to 6.
8. If `q_i != i`, we check the value of `sx_i`. If `sx_i == 0`, the number on index `q_i` is the largest. If `sx_i == 1`, the number on index `i` is the largest.


### explanation

After step 4, we'll quantum state ${1\over \sqrt{2}}\ket{0}\ket{a} + {1\over \sqrt{2}}\ket{1}\ket{b}$. After step 5 the state becomes ${1\over \sqrt{2}}\ket{0}\ket{a-X} + {1\over \sqrt{2}}\ket{1}\ket{b-X}$. After measurement, if we get the index register to be the same as the index we chose randoamly in step 2, then the we've got a number substracting itself on the data register. So we don't have any information on the comparision between two numbers. Hence when that happens, we have to repeat process 2 to 6 until we have `q_i != i`. When `q_i != i`, on the data register we get either `(a-b)` or `(b-a)`. And `sx_i` being `0` or `1` indicates `>` or `<`. Based on such information, we now know which number is the largest.

### analysis

- `q_i != i` happens with a probability $1\over 2$, so on average we need to run the quantum circuit twice to get the largest number.
- The number of qubit required for the algorithm is $O(\log(\max(a,b)))$ due to the implementation of qram.
- This algorithm works for all kinds of number. Because both the qram and quantum-classical adder work for all integers, and the success rate is always $1\over 2$, independent from the size of two integers.

### implementation


In [7]:
def find_the_largest_number(a, b, show_process: bool = False) -> int:

    num_list = [a,b]

    largest = None

    while True:
        i = random.randint(0, 1)
        X = num_list[i]

        # setup qram. 1 classical bit for storing sign bit
        qram_list = toQRAM(num_list, 1)
        
        # setup the query
        qQuery = QuantumCircuit(qram_list.qregs[0])
        qQuery.h(qQuery.qregs[0])
        
        # query on the qram
        qQuery = qram_list.compose(qQuery, qram_list.qregs[0], front=True)
        
        # apply add(-X)
        qQuery.append(qAdder(-X, qQuery.qregs[1].size), qQuery.qregs[1])

        qQuery.measure(qQuery.qregs[0], qQuery.cregs[0])
        qQuery.measure(qQuery.qregs[1][-1], qQuery.cregs[1])
        
        # run qc only once and get result
        res_query = execute(qQuery, simulator,shots=1).result().get_counts()
        # get sign bit and index bit
        res_sxi, res_qi= res_query.most_frequent().split(" ")

        # optional print
        if show_process:
            print(qQuery)
            # print("raw data: ", res_query)
            # print(f"index: {res_qi}, sign: {res_sxi}")

        # did not get the compare result 
        if int(res_qi) == i:
            continue

        if res_sxi == '0':
            largest = num_list[int(res_qi)]
        else:
            largest = num_list[i]

        break


    return largest

### some test cases

In [8]:
find_the_largest_number(5,-6, show_process=True)

        ┌───┐┌───────┐           ┌─┐   
   q_i: ┤ H ├┤0      ├───────────┤M├───
        └───┘│       │┌─────────┐└╥┘   
q_xi_0: ─────┤1      ├┤0        ├─╫────
             │       ││         │ ║    
q_xi_1: ─────┤2      ├┤1        ├─╫────
             │  QRAM ││         │ ║    
q_xi_2: ─────┤3      ├┤2 add(6) ├─╫────
             │       ││         │ ║    
q_xi_3: ─────┤4      ├┤3        ├─╫────
             │       ││         │ ║ ┌─┐
q_xi_4: ─────┤5      ├┤4        ├─╫─┤M├
             └───────┘└─────────┘ ║ └╥┘
   i: 1/══════════════════════════╩══╬═
                                  0  ║ 
  xi: 1/═════════════════════════════╩═
                                     0 


5

In [9]:
def test_find_the_largest_number(
    int_range: int,
    num_tests: int
):  
    """
        For two randomly generated integers in range `[-int_range, int_range]`, run the quantum algorithm vs the classical max function
        to confirm the result is correct. The test will be run for `num_tests` times.
    """
    print("Case a = b:")
    for i in range(num_tests):
        a = random.randint(-int_range, int_range)
        b = a

        qMax = find_the_largest_number(a,b)
        cMax = max(a,b)
        print(f"testcase-{i} [{a}, {b}]: qMax = {qMax}, cMax = {cMax}. Results agree: {qMax == cMax}")


    # random tests
    print("\nRandom test (a>b or a<b):")
    for i in range(num_tests):
        a = random.randint(-int_range, int_range)
        b = random.randint(-int_range, int_range)

        qMax = find_the_largest_number(a,b)
        cMax = max(a,b)
        print(f"testcase-{i} [{a}, {b}]: qMax = {qMax}, cMax = {cMax}. Results agree: {qMax == cMax}")

    return 

In [10]:
test_find_the_largest_number(128, 5)

Case a = b:
testcase-0 [44, 44]: qMax = 44, cMax = 44. Results agree: True
testcase-1 [-87, -87]: qMax = -87, cMax = -87. Results agree: True
testcase-2 [-58, -58]: qMax = -58, cMax = -58. Results agree: True
testcase-3 [-36, -36]: qMax = -36, cMax = -36. Results agree: True
testcase-4 [-105, -105]: qMax = -105, cMax = -105. Results agree: True

Random test (a>b or a<b):
testcase-0 [-112, 93]: qMax = 93, cMax = 93. Results agree: True
testcase-1 [-35, -35]: qMax = -35, cMax = -35. Results agree: True
testcase-2 [-74, -39]: qMax = -39, cMax = -39. Results agree: True
testcase-3 [90, 71]: qMax = 90, cMax = 90. Results agree: True
testcase-4 [-71, -119]: qMax = -71, cMax = -71. Results agree: True


## Conclusion

Our proposed quantum algorithm do solve the problem.

# Find the Largest Number (extented ver.)